In [19]:
# if u are using google colab, uncomment below
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# earth engine & map
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='geosoftware-sarflood')

In [3]:
# basemap
base_map = geemap.Map()
# selector
base_map.add('basemap_selector')
# add layer manager
base_map.add('layer_manager')
base_map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## Todo
### get the study boundaries of Kabupaten Demak
The data can be accessed from BIG [Badan Informasi Geografi](https://tanahair.indonesia.go.id/portal-web/) or using the global database that available in GEE.
For this project we'll use data from **FAO GAUL** Global administrative from GEE

In [4]:
# get the feature collections level 2 for regency/kabupaten 
GAUL_L2 = ee.FeatureCollection("FAO/GAUL/2015/level2")
# guerying to kabupaten demak
AOI_ee = GAUL_L2.filter(ee.Filter.eq('ADM2_NAME', 'Demak'));
# target study area
AOI_ee = AOI_ee.geometry() 
AOI_ee

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "FAO/GAUL/2015/level2"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.equals",
                "arguments": {
                  "leftField": {
                    "constantValue": "ADM2_NAME"
                  },
                  "rightValue": {
                    "constantValue": "Demak"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [5]:
# make the study area into a feature collection so that it can be exported into shapefile format
aoi_feature = ee.Feature(AOI_ee)
aoi_feature_collection = ee.FeatureCollection([aoi_feature])
aoi_feature_collection

In [6]:
# add AOI into GEEMAP's map
base_map.addLayer(AOI_ee, {}, "AOI Kabupaten Demak")
base_map.centerObject(AOI_ee, 10)
base_map

Map(center=[-6.917568625204991, 110.63566560202774], controls=(WidgetControl(options=['position', 'transparent…

## Todo
### Call the Sentinel-1 SAR from GEE
filtering and export data image, the data we're using Sentinel-1 SAR GRD (Ground Range Detected) level.

### Pre-processing
Sentinel-1 SAR GRD has pre-processed data so it can be use directly for further analysis. [Read this](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD)

### The Data
It have to 2 data: pre-flood and during the flood

In [7]:
# Create variable S1_Col to call Sentinel-1 SAR GRD Level image collection
S1_Col = ee.ImageCollection('COPERNICUS/S1_GRD')
S1_Col

### Pre-flood Sentinel-1 data

- Image collection data filtering process
- Filtering based on recording date range
- Filtering based on instrument mode, we will use Interferometric Wide Swath (IW)
- Filtering based on orbit properties, we will use Descending direction orbit
- Filtering based on study area
- Filtering based on band properties, we will select the VH and VV polarization bands

In [8]:
S1_filtered_Before = (S1_Col
               .filter(ee.Filter.date('2024-01-01', '2024-01-20'))  # filter date
               .filter(ee.Filter.eq('instrumentMode', 'IW'))  # filter instrument mode
               .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))  # filter orbit direction
               .filterBounds(aoi_feature_collection)  # filter study area
               .select(['VV', 'VH'])  # select band
               )
S1_filtered_Before

Create [composite median](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-median#colab-python) and clip based on Sentinel-1 SAR study area before flood

In [9]:
img_S1_Before = S1_filtered_Before.median().clip(AOI_ee)
img_S1_Before

### During-flood Sentinel-1 data

In [10]:
S1_filtered_After = (S1_Col
               .filter(ee.Filter.date('2024-02-05', '2024-02-09'))  # filter tanggal
               .filter(ee.Filter.eq('instrumentMode', 'IW'))  # filter instrument mode
               .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))  # filter arah orbit
               .filterBounds(aoi_feature_collection)  # filter wilayah kajian
               .select(['VV', 'VH'])  # pilih band
               )
S1_filtered_After

In [11]:
img_S1_After = S1_filtered_After.median().clip(AOI_ee)
img_S1_After

### Pre-processing step to apply reduce radar speckle using smoothing

In [12]:
# Apply reduce the radar speckle (noisy) by smoothing
smoothing_radius = 50 # meter, UN Spider
S1_smooth_before = img_S1_Before.focal_mean(smoothing_radius, 'circle', 'meters')
S1_smooth_after = img_S1_After.focal_mean(smoothing_radius, 'circle', 'meters')

## Visualizing Sentinel-1 SAR

In [13]:
base_map.addLayer(S1_smooth_before.select('VH'), {'min': -25, 'max': 5}, "Sentinel-1 SAR VH Before")
base_map.addLayer(S1_smooth_after.select('VH'), {'min': -25, 'max': 5}, "Sentinel-1 SAR VH After")
base_map.centerObject(AOI_ee, 10) #Set zoom in
base_map

Map(center=[-6.917568625204991, 110.63566560202774], controls=(WidgetControl(options=['position', 'transparent…

## Todo
### Download ESA World Land Cover Data
- [ESA World Land Cover](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200) (LC ESA) data is global land cover data that has several land cover classes with a spatial resolution of 10 m.
- We will use this LC ESA data for post-processing of the mapping results. We will utilize the permanent water body class to remove flood detection areas that overlap or are within permanent water bodies. This post-processing process allows us to minimize errors in flood mapping results.
- Furthermore, this LC ESA data will also be used for land cover analysis on the fourth day of the flood.

In [14]:
# Mednapatkan dataset ESA worldcover dan clip berdasarkan AOI
LC_dataset = ee.ImageCollection('ESA/WorldCover/v200').first().clip(AOI_ee);
LC_dataset

In [15]:
visualization = {'bands': ['Map'],};
base_map.centerObject(AOI_ee, 10) #Set zoom in
base_map.addLayer(LC_dataset, visualization, 'Landcover');
base_map

Map(center=[-6.917568625204991, 110.63566560202774], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
# Export data shapefile Batas Admin Kabupaten Demak
geemap.ee_to_shp(aoi_feature_collection, filename="./data/Kab_Demak.shp")

### Save to drive 
[click here](https://code.earthengine.google.com/tasks) to see the progress

In [17]:
image_data = [
    (S1_smooth_before, "S1_smooth_before"),
    (S1_smooth_after, "S1_smooth_after"),
    (LC_dataset, "LC_dataset")
]

for image, description in image_data:
    geemap.ee_export_image_to_drive(
        image,
        description=description,
        folder="Geosoftware_SARFLOOD_Case",
        region=AOI_ee,
        scale=10
    )